In [1]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [2]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [3]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)
context = InteractiveContext()

In [4]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-datauwirgjg2/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-06-21T16_03_03.758087-q4fecbi_/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1624305783,sum_checksum:1624305783"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [5]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /var/folders/l5/hhyn39nj4576qzvgqpr7thdw0000gp/T/tfx-interactive-2021-06-21T16_03_03.758087-q4fecbi_/CsvExampleGen/examples/1


In [6]:
import numpy as np
from tfx import v1 as tfx
from tfx.types import artifact_utils
from tfx.utils import io_utils
from tfx.components.util import tfxio_utils
from tfx.dsl.component.experimental.decorators import component
import apache_beam as beam
import random

# TODO: add optional schema and confirm that the ending dataset matches the schema's types
# Conversion from TFRecord -> dict -> TFRecord possibly inefficient; possible to keep TFRecord as string?
# having to find out the types again from the dict is possibly inefficient, schema is good
# worried about losing type information on conversion

@component
def UndersamplingComponent(
    examples: tfx.dsl.components.InputArtifact[tfx.types.standard_artifacts.Examples]):
        # examples = artifact_utils.get_single_instance(examples.outputs["examples"]._artifacts)
        tfxio_factory = tfxio_utils.get_tfxio_factory_from_artifact(examples=[examples], telemetry_descriptors=[])
        split_and_tfxio = [(split, tfxio_factory(io_utils.all_files_pattern(artifact_utils.get_split_uri([examples], split))))
                        for split in artifact_utils.decode_split_names(examples.split_names)]

        def generate_elements(data):
            for i in range(len(data[list(data.keys())[0]])):
                yield {key: data[key][i][0] if len(data[key][i]) > 0 else "" for key in data.keys()}

        def sample(key, value, side=0):
            for item in random.sample(value, side):
                yield item
                
        def convert_to_tfexample(data):
            features = dict()
            for key, val in data.items():
                if isinstance(val, (int, np.integer)):
                    features[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[data[key]]))
                elif isinstance(val, (float, np.inexact)):
                    features[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[data[key]]))
                elif isinstance(val, str):
                    features[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[str.encode(data[key])]))
                else:
                    features[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[data[key]]))
            return tf.train.Example(features=tf.train.Features(feature=features))
            
        split, tfxio = split_and_tfxio[0]
        # TODO: alter this code to account for multiple files in the uri and possibly different URI paths
        uri = examples.uri.split("/")
        uri[-3] = "UndersamplingComponent"
        uri.append(f"Split-{split}")
        uri = "/".join(uri)

        with beam.Pipeline() as p:
            data = (
                # TODO: convert to list and back using a schema to save key space?
                p 
                | 'TFXIORead[%s]' % split >> tfxio.BeamSource()
                | beam.Map(lambda x: x.to_pydict())
                | beam.FlatMap(generate_elements)
                | beam.Map(lambda x: (x["company"], x))
            )

            val = (
                data
                | beam.combiners.Count.PerKey()
                | beam.Values()
                | beam.CombineGlobally(lambda elements: min(elements or [-1]))
            )

            res = (
                data 
                | beam.GroupByKey()
                | beam.FlatMapTuple(sample, side=beam.pvalue.AsSingleton(val))
            )
            
            _ = (
                res
                | "ToTFExample" >> beam.Map(lambda x: convert_to_tfexample(x))
                | "Serialize" >> beam.Map(lambda x: x.SerializeToString())
                | "WriteToTFRecord" >> beam.io.tfrecordio.WriteToTFRecord(
                    uri,
                    shard_name_template="",
                    compression_type=beam.io.filesystem.CompressionTypes.GZIP)
            )
            
        example_artifact = output_dict[standard_component_specs.EXAMPLES_KEY].type()
        base_output_dir = os.path.join(pipeline_info.pipeline_root, component_info.component_id)

        example_artifact.uri = base_driver._generate_output_uri(
            base_output_dir, standard_component_specs.EXAMPLES_KEY, execution_id)
        update_output_artifact(exec_properties, example_artifact.mlmd_artifact)
        base_driver._prepare_output_paths(example_artifact)

In [7]:
undersampling = UndersamplingComponent(examples=example_gen.outputs['examples'])
context.run(undersampling)

INFO:absl:Running driver for UndersamplingComponent
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for UndersamplingComponent
INFO:absl:Running publisher for UndersamplingComponent
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: UndersamplingComponent
    execution_id: 2
    outputs:
)

NameError: name 'types' is not defined

NameError: name 'output_dict' is not defined